In [5]:
using Turing, Distributions
using PyPlot, PyCall

In [6]:
# Load transition matrix T and observations obs
include(Pkg.dir("Turing")*"/notebooks/data/hmmdemo.data.jl");

[PG]: Iter 1 out of 10


LoadError: LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{ForwardDiff.Dual{N,T<:Real}}, ::Array{Float64,1})
This may have arisen from a call to the constructor ForwardDiff.Dual{N,T<:Real}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  ForwardDiff.Dual{N,T}(::T, !Matched::ForwardDiff.Partials{N,T})
  ForwardDiff.Dual{N,A,B}(::A, !Matched::ForwardDiff.Partials{N,B})
  call{T}(::Type{T}, ::Any)
  ...
while loading /Users/kai/.julia/v0.4/Turing/notebooks/data/hmmdemo.data.jl, in expression starting on line 33
while loading In[6], in expression starting on line 2

In [3]:
K = 10
N = 51
initial = fill(1.0 / K, K)
means = collect(1.0:K)*5

@model hmmdemo begin
    states = tzeros(Int,N)
    # T = TArray{Array{Float64,}}
    
    # Prior over T
    # for i=1:K, @assume T[i,:] ~ Dirichlet(ones(K)./K); end
    
    @assume states[1] ~ Categorical(initial)
    for i = 2:N
        @assume states[i] ~ Categorical(vec(T[states[i-1],:]))
        @observe obs[i] ~ Normal(means[states[i]], 10)
    end
    @predict states
end

hmmdemo (generic function with 1 method)

In [4]:
chain = sample(hmmdemo, PG(100,100));

[PG]: Iter 1 out of 100


LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{ForwardDiff.Dual{N,T<:Real}}, ::Array{Float64,1})
This may have arisen from a call to the constructor ForwardDiff.Dual{N,T<:Real}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  ForwardDiff.Dual{N,T}(::T, !Matched::ForwardDiff.Partials{N,T})
  ForwardDiff.Dual{N,A,B}(::A, !Matched::ForwardDiff.Partials{N,B})
  call{T}(::Type{T}, ::Any)
  ...
while loading In[4], in expression starting on line 1

In [50]:
# Print the 1st sample trajectory
chain.value[1].value[:states]'

1x51 Array{Int64,2}:
 8  9  5  10  4  4  6  7  2  2  3  9  …  9  8  9  7  1  8  7  2  3  9  6  7

In [51]:
# plot in an external window since it doesn't work yet in IJulia
pygui(true)
# tell PyPlot that the plot is interactive
PyPlot.ion()
# . . . and that previous plots are overwritten
PyPlot.hold(false)
fig = gcf()
fig[:figsize]=(8,6)

for i=1:50
    # Plot trajectories
    plot(1:51, means[chain.value[i].value[:states]], 1:51, obs, "rs")
    
    # Then force the draw
    pause(0.05)
end